### Introduction to the EvidenceSeeker Pipeline

This notebook serves as an introductory example for configuring and running the pipeline using [dedicated endpoints on HuggingFace](https://huggingface.co/docs/inference-endpoints/index).

##### 0. For debugging purposes

In [4]:
# %pip pip install arize-phoenix
# %pip install llama-index-callbacks-arize-phoenix
# observability
import llama_index.core  # type: ignore
import phoenix as px  # type: ignore

px.launch_app()

llama_index.core.set_global_handler("arize_phoenix", endpoint="http://localhost:6006/v1/traces")

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


##### 1. Ensuring access to the models
We use HuggingFace's [Inference Endpoints](https://huggingface.co/docs/inference-endpoints/index) for model access. 
We use the following models:
- [Llama-3.1-70B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-70B-Instruct)
- [Paraphrase-multilingual-mpnet-base-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2)
Accessing the models used by the pipeline requires creating a dedicated endpoint and an access token.

For using your personal token, set up a file named `.env` with the following content: 
> `HF_TOKEN_EVIDENCE_SEEKER = <Your HuggingFace Token for Endpoint API Access>`.

Change the value of the variable `ENV_PATH` to the path of your `.env` file:

In [1]:
ENV_PATH = "../.env" #Change this value to the path to your .env file

And set the following variables to the url of your endpoint:

In [2]:
#Change this value to your endpoint url for Llama-3.1-70B-Instruct:
LLAMA_ENDPOINT_URL = "https://ev6086dt6s7nn1b5.us-east-1.aws.endpoints.huggingface.cloud"
LLAMA_MODEL = "meta-llama/Llama-3.2-3B-Instruct"

#Change this value to your endpoint url for Paraphrase-multilingual-mpnet-base-v2:
EMBED_ENDPOINT_URL = "https://ibpp4xgm0kspxkjb.us-east-1.aws.endpoints.huggingface.cloud"
EMBED_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

# HF Inference API, does not work:
# LLAMA_BASE_URL = "https://router.huggingface.co/hf-inference/v1/"
# EMBED_BASE_URL = "https://router.huggingface.co/hf-inference/v1/"

Then, run the following lines:

In [4]:
import dotenv
import os

if os.path.exists(ENV_PATH) and os.path.basename(ENV_PATH) == ".env":
    dotenv.load_dotenv(ENV_PATH)
    if "HF_TOKEN_EVIDENCE_SEEKER" in os.environ:
        print(f"HF_TOKEN_EVIDENCE_SEEKER environment variable found at {ENV_PATH}!")
    else: 
        raise Exception(f"Please set the HF_TOKEN_EVIDENCE_SEEKER environment variable in .env file at {ENV_PATH}")
else:
    raise Exception(f"Please set 'ENV_PATH' to a valid path to an '.env' file.")

HF_TOKEN_EVIDENCE_SEEKER environment variable found at ../.env!


##### 2. Ensuring access to the database

For simplicity, we will use the already built embedding included in this repository. It is built using articles from the journal "Aus Politik und Zeitgeschichte" (APuZ) provided by the Federal Agency for Civic Education (Bundeszentrale für politische Bildung/bpb).

In [6]:
EMBEDDING_PATH = "../TMP/APUZ/storage/index"

In [ ]:
from evidence_seeker.retrieval import RetrievalConfig
from evidence_seeker.preprocessing import ClaimPreprocessingConfig
from evidence_seeker.confirmation_analysis import ConfirmationAnalyzerConfig

retrieval_config = RetrievalConfig(
    index_persist_path=EMBEDDING_PATH,
    embed_base_url=EMBED_ENDPOINT_URL,
    embed_model_name=EMBED_MODEL
)

model = {"model_5" : {"name": "Llama-3.1-70B-Instruct",
                "description": "HF dedicated endpoint (debatelab)",
                "base_url": LLAMA_ENDPOINT_URL,
                "model": LLAMA_MODEL,
                "api_key_name": "HF_TOKEN_EVIDENCE_SEEKER",
                "backend_type": "tgi",
                "max_tokens": 2048,
                "temperature": 0.2,
            }
        }

preprocessing_config = ClaimPreprocessingConfig(used_model_key="model_5", models=model)

confirmation_analysis_config = ConfirmationAnalyzerConfig(used_model_key="model_5", models=model)

##### 3. Running the pipeline on a set of example inputs
We'll use the following example inputs for testing the pipeline:

In [8]:
example_inputs = [
    "Die Anzahl hybrider Kriege hat zugenommen.",
    "Die Osterweiterung hat die EU-Institutionen nachhaltig geschwächt.",
    "In der Bundeswehr gibt es keinen politischen Extremismus."
]

Run the following lines to instantiate an EvidenceSeeker. Note that it may take a few minutes to load the pre-built index. 

In [9]:
from evidence_seeker import EvidenceSeeker

evidence_seeker = EvidenceSeeker(retrieval_config=retrieval_config, preprocessing_config=preprocessing_config, confirmation_analysis_config=confirmation_analysis_config)

2025-03-10 15:33:56.019 | DEBUG    | evidence_seeker.backend:get_openai_llm:109 - Fetching api key via env var: HF_TOKEN_EVIDENCE_SEEKER
2025-03-10 15:33:56.025 | DEBUG    | evidence_seeker.backend:get_openai_llm:117 - Instantiating OpenAILike model (model: meta-llama/Llama-3.1-70B-Instruct,base_url: https://ev6086dt6s7nn1b5.us-east-1.aws.endpoints.huggingface.cloud).
2025-03-10 15:33:56.028 | INFO     | evidence_seeker.retrieval.base:load_index:95 - Loading index from disk at ../TMP/APUZ/storage/index
2025-03-10 15:35:56.336 | DEBUG    | evidence_seeker.backend:get_openai_llm:109 - Fetching api key via env var: HF_TOKEN_EVIDENCE_SEEKER
2025-03-10 15:35:56.340 | DEBUG    | evidence_seeker.backend:get_openai_llm:117 - Instantiating OpenAILike model (model: meta-llama/Llama-3.1-70B-Instruct,base_url: https://ev6086dt6s7nn1b5.us-east-1.aws.endpoints.huggingface.cloud).


The following lines will execute the pipeline on the given example inputs. Before running this cell, make sure the endpoint is running.

In [ ]:
from evidence_seeker import describe_result

results = []
for input in example_inputs:
    result = await evidence_seeker(input)
    results.append(result)
    md = describe_result(input, result)
    display(Markdown(md))
    display(Markdown("------\n"))

In [15]:
# For displaying all results at once

from IPython.display import display, Markdown

for (input, result) in zip(example_inputs, results):
    md = describe_result(input, result)
    display(Markdown(md))
    display(Markdown("------\n"))